# Notes

# Load Modules

In [3]:
import numpy as np
import pandas as pd

import anndata as adata
import scanpy as sc
import squidpy as sq
from SpaceFlow import SpaceFlow

# Load Data

In [ ]:
exp_df = pd.read_csv("simulated_expression.csv", index_col=0)
exp_df.head()

,cell1,cell2,cell3,cell4,cell5,cell6,cell7,cell8,cell9,cell10,...,cell991,cell992,cell993,cell994,cell995,cell996,cell997,cell998,cell999,cell1000
Burn1_TF1,2,2,8,6,5,2,7,1,3,4,...,4,2,3,6,2,4,2,3,8,4
Burn2_TF1,4,4,9,3,5,3,2,4,3,3,...,7,6,2,6,3,1,4,7,7,6
Burn3_TF1,4,1,6,3,2,3,5,2,4,4,...,8,5,5,2,3,5,4,6,7,2
Burn4_TF1,5,3,5,6,3,1,1,3,3,2,...,1,1,1,4,5,3,4,4,6,3
A1_TF1,2,4,8,4,1,2,4,3,2,2,...,5,1,2,2,3,3,3,4,5,0


In [ ]:
meta_info_df = pd.read_csv("full_simulation_data_with_noise.csv", index_col=0)
meta_info_df.head()

,Sample,UMAP1,UMAP2,time,lineage,Cell_Type,r,theta,x,y
Cell_ID,,,,,,,,,,
cell562,Simulation,3.141247,6.648345,0.325221,1,A,17.108442,92.387241,-0.712621,17.093594
cell779,Simulation,3.083348,6.590264,0.457619,1,A,20.294256,323.012103,16.210293,-12.209964
cell637,Simulation,3.039063,6.706306,0.459809,1,A,20.342765,184.818052,-20.270883,-1.708626
cell168,Simulation,2.943072,6.653493,0.491919,1,A,21.041083,250.849126,-6.902670,-19.876627
cell393,Simulation,3.054525,6.533715,0.491919,1,A,21.041083,162.404762,-20.056692,6.360523


In [4]:
ad = adata.AnnData(exp_df.T)

In [5]:
ad.X

array([[ 2,  4,  4, ..., 17,  6,  6],
       [ 2,  4,  1, ..., 13,  0,  7],
       [ 8,  9,  6, ..., 24,  4,  6],
       ...,
       [ 3,  7,  6, ..., 15,  1,  4],
       [ 8,  7,  7, ..., 15,  1, 10],
       [ 4,  6,  2, ..., 10,  1,  6]])

In [6]:
ad.obs_names

Index(['cell1', 'cell2', 'cell3', 'cell4', 'cell5', 'cell6', 'cell7', 'cell8',
       'cell9', 'cell10',
       ...
       'cell991', 'cell992', 'cell993', 'cell994', 'cell995', 'cell996',
       'cell997', 'cell998', 'cell999', 'cell1000'],
      dtype='object', length=1000)

In [7]:
ad.var_names

Index(['Burn1_TF1', 'Burn2_TF1', 'Burn3_TF1', 'Burn4_TF1', 'A1_TF1', 'A2_TF1',
       'A3_TF1', 'A4_TF1', 'A5_TF1', 'A6_TF1',
       ...
       'HK41', 'HK42', 'HK43', 'HK44', 'HK45', 'HK46', 'HK47', 'HK48', 'HK49',
       'HK50'],
      dtype='object', length=135)

In [8]:
ad.obs = ad.obs.join(meta_info_df)
ad.obs.head()

,Sample,UMAP1,UMAP2,time,lineage,Cell_Type,r,theta,x,y
cell1,Simulation,5.341630,-4.380950,990.257958,2,J,783.098137,127.503785,-476.760985,621.242027
cell2,Simulation,0.004179,0.829091,227.543106,1,D,452.535960,139.165465,-342.389193,295.902748
cell3,Simulation,-7.408302,-3.197001,558.623873,1,F,709.056758,349.044340,696.133880,-134.755730
cell4,Simulation,-7.239113,-3.128485,573.547864,1,F,718.465781,297.288026,329.390581,-638.509924
cell5,Simulation,-5.224359,-1.215822,500.117479,1,F,670.899196,24.532851,610.332666,278.567349


In [9]:
ad.obsm['spatial'] = ad.obs[['x', 'y']].values

# SpaceFlow

In [10]:
sf = SpaceFlow.SpaceFlow(adata=ad)

In [11]:
sf.preprocessing_data(n_top_genes=3000)

In [12]:
sf.train(spatial_regularization_strength=0.1, z_dim=50, lr=1e-3, epochs=1000, max_patience=50, min_stop=100, random_seed=42, gpu=0, regularization_acceleration=True, edge_subset_sz=1000000)

Epoch 2/1000, Loss: 3.408705234527588
Epoch 12/1000, Loss: 1.5153772830963135
Epoch 22/1000, Loss: 1.3853490352630615
Epoch 32/1000, Loss: 1.3262304067611694
Epoch 42/1000, Loss: 1.2307413816452026
Epoch 52/1000, Loss: 1.1209650039672852
Epoch 62/1000, Loss: 0.9460107684135437
Epoch 72/1000, Loss: 0.8356530666351318
Epoch 82/1000, Loss: 0.744941234588623
Epoch 92/1000, Loss: 0.6351093053817749
Epoch 102/1000, Loss: 0.5351232886314392
Epoch 112/1000, Loss: 0.5140056610107422
Epoch 122/1000, Loss: 0.5894868969917297
Epoch 132/1000, Loss: 0.5158069133758545
Epoch 142/1000, Loss: 0.43827372789382935
Epoch 152/1000, Loss: 0.4136343002319336
Epoch 162/1000, Loss: 0.3894919157028198
Epoch 172/1000, Loss: 0.35410675406455994
Epoch 182/1000, Loss: 0.32037344574928284
Epoch 192/1000, Loss: 0.3028799295425415
Epoch 202/1000, Loss: 0.3410320281982422
Epoch 212/1000, Loss: 0.2946223318576813
Epoch 222/1000, Loss: 0.3160688579082489
Epoch 232/1000, Loss: 0.3033824861049652
Epoch 242/1000, Loss: 0.28

array([[ 3.6518886 ,  3.644271  ,  2.056975  , ...,  2.1606865 ,
         2.8216217 , -1.2302538 ],
       [ 1.8339579 ,  2.448432  ,  1.9350927 , ...,  1.5817999 ,
        -0.06176059, -0.66618675],
       [ 3.4309165 ,  3.0622802 ,  1.7470442 , ...,  2.3560681 ,
         1.710612  , -1.0808235 ],
       ...,
       [ 3.0763502 ,  2.8791084 ,  2.0787172 , ...,  2.1463373 ,
         1.0361824 , -1.0748554 ],
       [ 5.1107574 ,  4.197751  ,  2.663534  , ...,  2.6584816 ,
         2.2861319 , -1.3519484 ],
       [ 4.9640937 ,  4.5026555 ,  2.7537236 , ...,  2.5354414 ,
         2.5385947 , -1.405331  ]], dtype=float32)

In [17]:
sf.segmentation(domain_label_save_filepath="./domains.tsv", n_neighbors=50, resolution=1.0)

Performing domain segmentation
Segmentation complete, domain labels of cells or spots saved at ./domains.tsv !


In [18]:
sf.plot_segmentation(segmentation_figure_save_filepath="./domain_segmentation.pdf", colormap="tab20", scatter_sz=1., rsz=4., csz=4., wspace=.4, hspace=.5, left=0.125, right=0.9, bottom=0.1, top=0.9)

Plotting complete, segmentation figure saved at ./domain_segmentation.pdf !


In [19]:
sf.pseudo_Spatiotemporal_Map(pSM_values_save_filepath="./pSM_values.tsv", n_neighbors=20, resolution=1.0)

Performing pseudo-Spatiotemporal Map
pseudo-Spatiotemporal Map(pSM) calculation complete, pSM values of cells or spots saved at ./pSM_values.tsv!


In [20]:
sf.plot_pSM(pSM_figure_save_filepath="./pseudo-Spatiotemporal-Map.pdf", colormap="roma", scatter_sz=1., rsz=4., csz=4., wspace=.4, hspace=.5, left=0.125, right=0.9, bottom=0.1, top=0.9)

Plotting complete, pseudo-Spatiotemporal Map figure saved at ./pseudo-Spatiotemporal-Map.pdf !


In [2]:
import session_info

In [4]:
session_info.show()